<a href="https://colab.research.google.com/github/awaiskhan005/DATA-SCIENCE-AND-AI-/blob/main/btc_continued.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Function to fetch data
def fetch_data(ticker):
    data = yf.download(ticker, period='1d', interval='1m')
    return data

In [2]:
# Function to normalize data
def normalize_data(data):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data)
    return scaled_data, scaler

In [6]:
# Function to prepare LSTM data
def prepare_lstm_data(data, window_size=60):
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i])
        y.append(data[i, 0])
    X, y = np.array(X), np.array(y)
    return X, y

In [7]:
# Function to create LSTM model
def create_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [8]:
# Fetching data
btc_data = fetch_data('BTC-USD')

[*********************100%%**********************]  1 of 1 completed


In [13]:
!pip install TA-Lib==0.4.24


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for TA-Lib
  Running setup.py clean for TA-Lib
Failed to build TA-Lib
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (TA-Lib)


In [15]:
pip install pandas_ta


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 977.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218910 sha256=2448d6beb0e044296a5df10482c239f1a37e37fac01de7cea4546cf51affa6f0
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [17]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta

# Fetching BTC-USD data
btc_data = yf.download('BTC-USD', period='max', interval='1d')

# Adding indicators
btc_data['RSI'] = ta.rsi(btc_data['Close'])

# Handle multiple columns returned by ta.stochrsi
stoch_rsi = ta.stochrsi(btc_data['Close'])
btc_data['stoch_RSI_k'] = stoch_rsi['STOCHRSIk_14_14_3_3']
btc_data['stoch_RSI_d'] = stoch_rsi['STOCHRSId_14_14_3_3']

# Handle multiple columns returned by ta.bbands
bbands = ta.bbands(btc_data['Close'])
btc_data['BB_upper'] = bbands['BBU_5_2.0']
btc_data['BB_lower'] = bbands['BBL_5_2.0']

btc_data['EMA_Cross'] = ta.ema(btc_data['Close'])

# Display the DataFrame to verify the indicators
print(btc_data.head())


[*********************100%%**********************]  1 of 1 completed

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2014-09-17  465.864014  468.174011  452.421997  457.334015  457.334015   
2014-09-18  456.859985  456.859985  413.104004  424.440002  424.440002   
2014-09-19  424.102997  427.834991  384.532013  394.795990  394.795990   
2014-09-20  394.673004  423.295990  389.882996  408.903992  408.903992   
2014-09-21  408.084991  412.425995  393.181000  398.821014  398.821014   

              Volume  RSI  stoch_RSI_k  stoch_RSI_d   BB_upper    BB_lower  \
Date                                                                         
2014-09-17  21056800  NaN          NaN          NaN        NaN         NaN   
2014-09-18  34483200  NaN          NaN          NaN        NaN         NaN   
2014-09-19  37919700  NaN          NaN          NaN        NaN         NaN   
2014-09-20  36863600  NaN          NaN          NaN        NaN         NaN   
2014-09-21  2

In [18]:
# Handling missing values
btc_data.fillna(method='ffill', inplace=True)

<ipython-input-18-960e8199d826>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  btc_data.fillna(method='ffill', inplace=True)


In [19]:
# Normalizing data
btc_data_normalized, btc_scaler = normalize_data(btc_data[['Close', 'RSI', 'stoch_RSI', 'BB_upper', 'BB_lower', 'EMA_Cross']])

KeyError: "['stoch_RSI'] not in index"

In [20]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
from sklearn.preprocessing import MinMaxScaler

# Fetching BTC-USD data
btc_data = yf.download('BTC-USD', period='max', interval='1d')

# Adding indicators
btc_data['RSI'] = ta.rsi(btc_data['Close'])

# Handle multiple columns returned by ta.stochrsi
stoch_rsi = ta.stochrsi(btc_data['Close'])
btc_data['stoch_RSI_k'] = stoch_rsi['STOCHRSIk_14_14_3_3']
btc_data['stoch_RSI_d'] = stoch_rsi['STOCHRSId_14_14_3_3']

# Handle multiple columns returned by ta.bbands
bbands = ta.bbands(btc_data['Close'])
btc_data['BB_upper'] = bbands['BBU_5_2.0']
btc_data['BB_lower'] = bbands['BBL_5_2.0']

btc_data['EMA_Cross'] = ta.ema(btc_data['Close'])

# Normalizing data
features_to_normalize = ['Close', 'RSI', 'stoch_RSI_k', 'stoch_RSI_d', 'BB_upper', 'BB_lower', 'EMA_Cross']
btc_data_normalized = btc_data[features_to_normalize].dropna()

btc_scaler = MinMaxScaler()
btc_data_normalized[features_to_normalize] = btc_scaler.fit_transform(btc_data_normalized)

# Display the normalized DataFrame to verify
print(btc_data_normalized.head())


[*********************100%%**********************]  1 of 1 completed

               Close       RSI  stoch_RSI_k  stoch_RSI_d  BB_upper  BB_lower  \
Date                                                                           
2014-10-18  0.002926  0.504687     0.837158     0.865407  0.002467  0.003154   
2014-10-19  0.002900  0.494887     0.854259     0.846639  0.002377  0.003186   
2014-10-20  0.002808  0.460249     0.826154     0.839190  0.002321  0.003177   
2014-10-21  0.002858  0.479646     0.759779     0.813397  0.002320  0.003201   
2014-10-22  0.002813  0.461447     0.647173     0.744369  0.002322  0.003196   

            EMA_Cross  
Date                   
2014-10-18   0.002285  
2014-10-19   0.002306  
2014-10-20   0.002306  
2014-10-21   0.002315  
2014-10-22   0.002314  


In [21]:
# Preparing LSTM data
X, y = prepare_lstm_data(btc_data_normalized)

# Splitting data into training and testing sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

KeyError: (60, 0)